In [1]:
from pyspark import SparkContext
#from dummy_spark import SparkContext

from sympy import *
from drudge import *
from gristmill import *
from agp_fermi import *

In [2]:
fort_printer = FortranPrinter()

In [3]:
ctx = SparkContext()
dr = AGPFermi(ctx)

In [4]:
nam = dr.names

In [5]:
c_ = nam.c_
c_dag = nam.c_dag
P_ = nam.P_
P_dag = nam.P_dag
N_ = nam.N_
Sp = nam.J_p
Sm = nam.J_m
Sz = nam.J_z

In [6]:
p, q, r, s  = nam.A_dumms[:4]

In [7]:
h = IndexedBase('h')
w = IndexedBase('w')
v = IndexedBase('v')
K = IndexedBase('K')
t = IndexedBase('t')
z = IndexedBase('z')
u = IndexedBase('u')
x = IndexedBase('x')
y = IndexedBase('y')

In [8]:
G = Symbol('G')

In [9]:
Np = 2*y[p]**2 + 2*x[p]*y[p]*(P_dag[p]+P_[p]) + (x[p]**2-y[p]**2)*N_[p]
Nq = 2*y[q]**2 + 2*x[q]*y[q]*(P_dag[q]+P_[q]) + (x[q]**2-y[q]**2)*N_[q]
Pdp = x[p]*y[p] + x[p]**2*P_dag[p] - y[p]**2*P_[p] - x[p]*y[p]*N_[p]
Pq = x[q]*y[q] + x[q]**2*P_[q] - y[q]**2*P_dag[q] - x[q]*y[q]*N_[q]

In [14]:
pa_range = dr.names.A
term = dr.sum((p,pa_range),(q,pa_range),Np*Nq)

In [16]:
tmp= term.simplify().merge()

In [24]:
tmp.filter(lambda x: len(x.vecs)==2).display()

<IPython.core.display.Math object>

In [10]:
dr.set_symm(w,Perm([1,0],IDENT),valence=2)
#dr.set_symm(v,Perm([1,0],IDENT),valence=2)
pa_range = dr.names.A
#ham = dr.sum((p,pa_range),h[p]*Np) + dr.sum((p,pa_range),(q,pa_range),w[p,q]*Np*Nq)/4 + dr.sum((p,pa_range),(q,pa_range),v[p,q]*Pdp*Pq)
ham = dr.sum((p,pa_range),h[p]*Np) - G*dr.sum((p,pa_range),(q,pa_range),Pdp*Pq)

In [11]:
H = ham.simplify().merge().map2amps(factor)

In [12]:
H.display()

<IPython.core.display.Math object>

In [13]:
exprlist = []

In [13]:
import numpy as np
def get_vev_of_term(term):
    return [Term(sums=(), amp=term.amp, vecs=())]
#    nsum = len(term.sums)
#    if (nsum == 1):
#        return [Term(sums=(), amp=term.amp, vecs=())]
#    if (nsum == 2):
#        return [Term(sums=[(q, pa_range)], amp=term.amp, vecs=())]
#    return [Term(sums=term.sums, amp=term.amp, vecs=())]

In [22]:
def getterms(term):
    vecs = term.vecs
#    if (len(vecs) == 0):
#    if (len(vecs) == 1 and vecs[0].label == 'P'):
    if (len(vecs) == 2 and vecs[0].label == 'N' and vecs[1].label == 'P'):
        return True
    else:
        return False

In [21]:
H.subst(w[p,p],0).filter(lambda x: getterms(x)).merge().map2amps(factor).display()

<IPython.core.display.Math object>

In [23]:
H.subst(w[p,p],0).filter(lambda x: getterms(x)).merge().map2amps(factor).display()

<IPython.core.display.Math object>

In [102]:
equ = H.subst(w[p,p],0).filter(lambda x: getterms(x)).bind(get_vev_of_term)

In [103]:
equ.display()

<IPython.core.display.Math object>

In [104]:
H20 = IndexedBase('H20')
exprH20 = dr.define(H20[p],equ)

In [106]:
exprH11.display()

<IPython.core.display.Math object>

In [99]:
exprH02.display()

<IPython.core.display.Math object>

In [109]:
eval_equ = optimize(
    [exprH00,exprH20,exprH11,exprH02,exprH40,exprH31,exprH22,exprHb22,exprH13,exprH04],
    interm_fmt='tau{}',drop_cutoff=2
)

/projects/guscus/apps-python/drudge/gristmill/build/lib.linux-x86_64-3.6/gristmill/optimize.py:1985: UserWarning: Internal deficiency: summation intermediate may not be fully canonicalized
  'Internal deficiency: '


In [110]:
evals = fort_printer.doprint(eval_equ)
with open('Hub2Pair.f90', 'w') as fp:
    fp.write(evals)

In [ ]:
Z = dr.ei

In [107]:
expr = dr.define(H40[p,r,q,s],u[p,q,r,s])

In [108]:
py_printer = EinsumPrinter()

In [109]:
print(py_printer.doprint([expr]))

    H40 = zeros((M_orb, M_orb, M_orb, M_orb))

    H40 += einsum(
        "pqrs->prqs", u
    )



In [32]:
expr1 = dr.sum(P_dag[p]+v[p]*(1-N_[p])-v[p]**2*P_[p])
expr2 = dr.sum(P_dag[q]+v[q]*(1-N_[q])-v[q]**2*P_[q])
expr3 = dr.simplify(expr1*expr2).merge()

In [37]:
expr3.display()

<IPython.core.display.Math object>

In [35]:
def getterms(term):
    vecs = term.vecs
    if (len(vecs)==0):
        return True
    elif (vecs[-1].label == 'P' or vecs[-1].label == 'N'):
        return False
    else:
        return True

In [36]:
expr3.filter(lambda x: getterms(x)).display()

<IPython.core.display.Math object>

In [10]:
v1 = dr.sum(P_[p])
n =t* dr.sum(2*u[q]*v[q]*P_dag[p] + 2*u[p]*v[p]*P_[p] + (u[p]**2-v[p]**2)*(1-N_[p]))

In [13]:
v2=(v1|n).simplify()
v3=(v2|n/2).simplify()
v4=(v3|n/3).simplify()
v5=(v4|n/4).simplify()

In [10]:
S2 = (Sp[p]*Sm[q]+Sm[p]*Sp[q])/2+Sz[p]*Sz[q]
Hp = dr.einst(h[p]*N_[p])+dr.einst(w[p,q]*N_[p]*N_[q])/4+dr.einst(v[p,q]*P_dag[p]*P_[q])
Hs = dr.einst(K[p,q]*S2)

In [13]:
ham = dr.simplify(Hp)

In [25]:
Z1 = dr.einst(z[p]*P_[p])
T1 = dr.einst(t[p]*P_dag[p])
T2 = dr.einst(t[p,q]*P_dag[p]*P_dag[q])
T = T1 + T2

In [28]:
h0 = ham
h1 = (h0|-Z1).simplify()
h2 = (h1|-Z1/2).simplify()

In [29]:
Hz = dr.simplify(h0+h1+h2)

In [31]:
H0 = Hz
H1 = (H0|T).simplify()
H2 = (H1|T/2).simplify()
H3 = (H2|T/3).simplify()
H4 = (H3|T/4).simplify()

In [34]:
Hbar = dr.simplify(H0+H1+H2+H3+H4)

In [96]:
C = dr.einst(t[p,q]*P_[p]*N_[q])
Z = dr.einst(z[p]*P_[p])
C1 = (C|Z).simplify()
C2 = (C1|Z).simplify()
C3 = (C2|Z).simplify()
C4 = (C3|Z).simplify()

In [97]:
dr.simplify(C+C1+C2+C3+C4).merge().map2amps(factor).display()

<IPython.core.display.Math object>

In [98]:
C = dr.einst(t[p,q]*N_[p]*N_[q])
Z = dr.einst(z[p]*P_[p])
C1 = (C|Z).simplify()
C2 = (C1|Z).simplify()
C3 = (C2|Z).simplify()
C4 = (C3|Z).simplify()

In [99]:
dr.simplify(C+C1+C2+C3+C4).merge().map2amps(factor).display()

<IPython.core.display.Math object>

In [90]:
C = dr.einst(t[p,q]*P_dag[p]*N_[q])
Z = dr.einst(z[p]*P_[p])
C1 = (C|Z).simplify()
C2 = (C1|Z).simplify()
C3 = (C2|Z).simplify()
C4 = (C3|Z).simplify()

In [91]:
dr.simplify(C+C1+C2+C3+C4).merge().map2amps(factor).display()

<IPython.core.display.Math object>

In [92]:
C = dr.einst(t[p,q]*P_dag[p]*P_[q])
Z = dr.einst(z[p]*P_[p])
C1 = (C|Z).simplify()
C2 = (C1|Z).simplify()
C3 = (C2|Z).simplify()
C4 = (C3|Z).simplify()

In [93]:
dr.simplify(C+C1+C2+C3+C4).merge().map2amps(factor).display()

<IPython.core.display.Math object>

In [101]:
C = dr.einst(t[p,q]*P_dag[p]*P_dag[q])
Z = dr.einst(z[p]*P_[p])
C1 = (C|Z).simplify()
C2 = (C1|Z/2).simplify()
C3 = (C2|Z/3).simplify()
C4 = (C3|Z/4).simplify()

In [102]:
dr.simplify(C+C1+C2+C3+C4).merge().map2amps(factor).display()

<IPython.core.display.Math object>

In [127]:
term1 = dr.sum(P_dag[p]*N_[p]).local_terms[0]
v1 = term1.vecs

In [128]:
ind=v1[0].base

In [129]:
v1[0].indices

(p,)

In [46]:
import numpy as np
def get_vev_of_term(term):
    vecs = term.vecs
    inds = np.ravel([i.indices[1] for i in vecs])
    label = get_spin(spins)
    Y1 = IndexedBase(label)
    D = IndexedBase('delta')
    if (spin0(vecs) == 1):
        return [Term(sums=term.sums, amp=0 * Y1[inds], vecs=())]
    else:
        return [Term(sums=term.sums, amp=term.amp * Y1[inds], vecs=())]

In [103]:
dr.sum(c_[p]*c_[q]*c_dag[r]*c_dag[s]).simplify().display()

<IPython.core.display.Math object>